In [ ]:
import collections
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.compat.v1.keras.layers import CuDNNGRU
from tensorflow.keras.layers import GRU, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Data Loading and Setup

In [2]:
# Load in train and test data to pandas dataframe
xy_train_df = pd.read_csv('xy_train.csv')
x_test_df = pd.read_csv('x_test.csv')

In [3]:
# Remove incorrectly labelled data
xy_train_df.drop(xy_train_df[xy_train_df.label > 1].index, inplace=True)
xy_train_df.reset_index(inplace=True)

In [4]:
xy_train_df.shape
# Should be (59768, 4)

(59768, 4)

# Preprocessing

In [5]:
# Separate out the training data and labels
x = xy_train_df.text
y = xy_train_df.label

In [6]:
# Remove English stopwords
def removeStopEnglish(list_of_text):
    # Get list of English words to remove
    stop = stopwords.words('english')
    # Set all input text to lowercase for string comparison
    list_of_text = list_of_text.str.lower()
    # Removing all English stopwords from input text 
    # (Source for this lambda function in report)
    list_of_text_nostop = list_of_text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    # Return updated list
    return list_of_text_nostop

x = removeStopEnglish(x)

In [ ]:
x.head()

In [7]:
# Maximum number of words to keep based on word frequency (variable for Tokenizer)
vocab_size = 40000

# Build vocabulary from training set
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(x)

# Preprocessing function
def padText(list_of_text, max_len):
    # Transforms list of sequences into 2D numpy array
    # Sequences shorter than maxlen are padded at the end ('post' argument)
    # Sequences longer than maxlen are truncated 
    return pad_sequences(
        tokenizer.texts_to_sequences(list_of_text),
        maxlen=max_len,
        padding='post',
    )
# Length to make all sequences in vocabulary (variable for pad_sequences)
max_len = 40  
# padding is done inside: 
x = padText(x, max_len)


In [8]:
# Split the training data into train and validate, 20% validation at beginning
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2)

print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)

(47814, 40) (47814,)
(11954, 40) (11954,)


In [14]:
# Display last 5 entries with pretty printer
pprint(tokenizer.sequences_to_texts(x_train[:5]))

['pregnant escaped las vegas concert shooting couple welcomed baby world',
 'church polish tourist place full inside people wait outside listening '
 'preacher',
 "scott confirms negan's kill premiere confirms premiere show killed we've "
 'learned nothing new',
 'burrs outback central australia size lifesaver affectionately known teddy '
 "bear's",
 '25 000 pounds honey creates sticky situation new zealand terror attack '
 'denied bail']


In [15]:
print('total words in the dictionary', tokenizer.num_words)

total words in the dictionary 40000


# Models

## Fully Connected Models

In [ ]:
# MODEL 1
# Version 1: No stopword removal
# Version 2: English stopwords removed
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]
# Reduce embedded back to 2D by taking the mean along axis 1 
averaged = tf.reduce_mean(embedded, axis=1) # [None, 100]
# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(averaged) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='functional_1_1'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 20
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1)

In [ ]:
# MODEL 2
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]
# Reduce embedded back to 2D by taking the mean along axis 1 
averaged = tf.reduce_mean(embedded, axis=1) # [None, 100]
# Fully connected layer
x = keras.layers.Dense(16)(averaged) # [None, 16]
x = keras.layers.Dropout(0.5)(x) # [None, 16]
x = keras.layers.Dense(8)(x) # [None, 8]
x = keras.layers.Dropout(0.5)(x) # [None, 8]
# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='fully_connected_2_2'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 20
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1)

## Recurrent GRU Models

Simple Model

In [ ]:
# GRU model 1
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]

x = GRU(75)(embedded) # [None, 75]
# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='gru_1'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 5
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1)

In [ ]:
# GRU simple model 2
# Single layer, smaller number of units
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]

x = GRU(4)(embedded) # [None, 4]

# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='gru_simple_2'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 20
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1,
                    callbacks=keras.callbacks.EarlyStopping())

Multi-layer GRU

In [ ]:
# GRU model 2 - Multi layer
# Added second GRU layer
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]

x = GRU(75, return_sequences=True)(embedded) # [None, 100, 75]
x = GRU(75)(x) # [None, 75]

# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='gru_2'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 5
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1)

Multi-Layer GRU #2

In [ ]:
# GRU model 3 - Multi Layer
# Changed layer unit sizes
# Adding callbacks EarlyStopping in fit
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]

x = GRU(16, return_sequences=True)(embedded) # [None, 100, 75]
x = GRU(8)(x) # [None, 75]

# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='gru_3'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 20
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1,
                    callbacks=keras.callbacks.EarlyStopping())

Bidirectional GRU

In [ ]:
# GRU Model 4 - Bidirectional, single layer 
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]

x = Bidirectional(GRU(4))(embedded) # [None, 8]

# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='gru_bidirectional_1'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)


epochs = 20
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1,
                    callbacks=keras.callbacks.EarlyStopping())

Model: "gru_bidirectional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 40)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 40, 100)           4000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 8)                 2544      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 4,002,553
Trainable params: 4,002,553
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# GRU Model 4 - Bidirectional, multi-layer 
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]

x = Bidirectional(GRU(4))(embedded) # [None, 8]

# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='gru_bidirectional_1'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 20
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1,
                    callbacks=keras.callbacks.EarlyStopping())

## LSTM Models

Simple Model 1

In [ ]:
# LSTM Model 1
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]

x = LSTM(4)(embedded) # [None, 4]

# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='lstm_1'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 10
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1,
                    callbacks=keras.callbacks.EarlyStopping())

Multi-layer Model

In [ ]:
# LSTM Model 2 - Multi-layer
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]

x = LSTM(32, return_sequences=True)(embedded) # [None, max_len, 32]
x = LSTM(16, return_sequences=True)(x) # [None, max_len, 16]
x = LSTM(8, return_sequences=True)(x) # [None, max_len, 8]
x = LSTM(4)(x) # [None, 4]

# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='lstm_2'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(learning_rate=0.0001),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 20
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1,
                    callbacks=keras.callbacks.EarlyStopping())

Simple Model 2

In [ ]:
# LSTM Model 3
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]

x = LSTM(4)(embedded) # [None, 4]

# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='lstm_3'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(learning_rate=0.0001),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 20
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1,
                    callbacks=keras.callbacks.EarlyStopping())

Bidirectional Model

In [ ]:
# LSTM Bidirectional Model 1 
# No early stopping, single bidirectional layer
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]

x = Bidirectional(LSTM(4))(embedded) # [None, 8]

# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='lstm_bidirectional_1'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 10
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1,)

In [ ]:
# LSTM Bidirectional Model 2 
# Same as model 1, using decreased learning rate
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]

x = Bidirectional(LSTM(4))(embedded) # [None, 8]

# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='lstm_bidirectional_2'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(learning_rate=0.0001),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 15
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1,)
                    # callbacks=keras.callbacks.EarlyStopping())

In [ ]:
# LSTM Bidirectional Multilayer Model
# No early stopping, multi bidirectional layer
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(seq_in) # [None, max_len, 100]

x = Bidirectional(LSTM(8, return_sequences=True))(embedded) # [None, max_len, 16]
x = Bidirectional(LSTM(4))(x) # [None, 8]

# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='lstm_bidirectional_3'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(learning_rate=0.0001),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 10
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1,)

In [ ]:
# LSTM Model 1
# Input layer for neural network
seq_in = keras.Input(batch_shape=(None, max_len)) # [None, max_len]
# Layer that turns positive integers into dense vectors of fixed size
embedded = keras.layers.Embedding(tokenizer.num_words, 200)(seq_in) # [None, max_len, 100]

x = LSTM(8)(embedded) # [None, 4]

# Output layer, sigmoid activation
pred = keras.layers.Dense(1, activation='sigmoid')(x) # [None, 1]

# Group layers together into model
model = keras.Model(
    inputs=seq_in,
    outputs=pred,
    name='lstm_1_3'
)

# Configure model for training
model.compile(
    # Optimize using Adam 
    optimizer=Adam(),
    # Value to be minimized by the model
    loss='binary_crossentropy',
    # Evaluate model with these metrics (Kaggle using AUC)
    metrics=['accuracy', 'AUC']
)

epochs = 10
batch_size = 64
# Fit model to training data
history = model.fit(x_train,
                    y_train,
                    # Number of iterations over entire x and y data
                    epochs=epochs,
                    # Number of samples per gradient update
                    batch_size=batch_size,
                    # Data to test model on
                    validation_data=(x_valid, y_valid),
                    # Shows progress bar for each epoch
                    verbose=1,)
                    # callbacks=keras.callbacks.EarlyStopping())

# Model Analysis

In [ ]:
model.summary()

In [ ]:
# Create AUC metric vs. epoch plot
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('model accuracy - ' + model.name)
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Output

In [13]:
# Remove English stop words from test set
x_test = removeStopEnglish(x_test_df.text)
# Process test set with padding/truncating to maxlen
x_test = padText(x_test, max_len)
# Run test set through model
y_predict = np.squeeze(model.predict(x_test))

# Setting up string for output file name
output_name = model.name + '_' + str(epochs) + "_" + str(batch_size) + '.csv'

# Put ID back together with prediction and save out for submission
pd.DataFrame(
    {'id': x_test_df.index,
     'label': y_predict}).to_csv(output_name, index=False)